In [ ]:
!pip install ultralytics
!pip install shutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 10.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
  Created wheel for shutils: filename=shutils-0.1.0-py3-none-any.whl size=3274 sha256=dd4d14db2ddc0f106feb2f9060bf09d40f3563f1a5138f7094a6ba82b59859d6
  Stored in directory: /root/.cache/pip/wheels/62/63/04/81e549bdb44792d8b62938cffc3bd00a34addabe1da3693db8
Successfully built shutils


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import albumentations as A
from imblearn.over_sampling import RandomOverSampler
from ultralytics import YOLO
import numpy as np
from shutil import rmtree

In [ ]:
DATASET = Path('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet')
#DATASET = Path('drive/MyDrive/Models/TurbinesDetector/Dataset')
DATASET.mkdir(parents=True, exist_ok=True)
DATASET_TIERS = DATASET.parent

In [ ]:
!unzip -joq 'drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/Turbines.zip' -d 'drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet'

In [ ]:
imgs = [file for file in DATASET.iterdir()]   # Соберём картинки в датасет.
print(f"Всего изображений: {len(imgs)}")


Всего изображений: 2886


In [ ]:
tier_size = [30, 300,]    # размерность 30 и 300
tier_size = np.cumsum(tier_size)
tier_size

array([ 30, 330])

In [ ]:
np.random.shuffle(imgs)   # Перемешаем датасет с картинками
imgs[:5]    # Посмотрим

[PosixPath('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/turbine_1106.jpg'),
 PosixPath('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/turbine_1823.jpg'),
 PosixPath('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/turbine_2684.jpg'),
 PosixPath('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/turbine_2689.jpg'),
 PosixPath('drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/DataSet/turbine_648.jpg')]

In [ ]:
img_tiers = np.split(imgs, tier_size) #Разбиваем массив на 3 части
print('Длина первой части = ', len(img_tiers[0]))
print('Длина второй части = ', len(img_tiers[1]))
print('Длина третьей части = ', len(img_tiers[2]))

Длина первой части =  30
Длина второй части =  300
Длина третьей части =  2556


In [ ]:
for i, tier in enumerate(img_tiers):
    path_to_save = DATASET_TIERS.joinpath(f'Tier_{i+1}/images')
    path_to_save.mkdir(exist_ok=True, parents=True)
    for img in tqdm(tier):
        path_to_save.joinpath(img.name).write_bytes(img.read_bytes())



100%|██████████| 2556/2556 [07:41<00:00,  5.54it/s]


# Работа с частью 1. 30 картинок

In [ ]:
DATASET_TIER_1 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_1')
#DATASET_TIER_1 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_1')

In [ ]:
imgs = [img.as_posix() for img in DATASET_TIER_1.joinpath('images').iterdir()]
len(imgs)

59

In [ ]:
train, val = train_test_split(imgs, test_size=0.2)
print(f"Train: {len(train)}, Val: {len(val)}")


Train: 47, Val: 12


In [ ]:
DATASET_TIER_1.joinpath('train.txt').write_text('\n'.join(train))
DATASET_TIER_1.joinpath('val.txt').write_text('\n'.join(val))

1172

In [ ]:
dataset_yaml = fr'''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: {DATASET_TIER_1}  # dataset root dir
train: train.txt  # train images (relative to 'path') 128 images
val: val.txt  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: Turbine
'''
DATASET_TIER_1.joinpath('dataset.yaml').write_text(dataset_yaml)

409

### Обучение модели

In [ ]:
MODEL_TIER_1 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_1')
#MODEL_TIER_1 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Models/Tier_1')
MODEL_TIER_1.mkdir(parents=True, exist_ok=True)

In [ ]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 141MB/s] 


In [ ]:
model.train(
    data=DATASET_TIER_1.joinpath('dataset.yaml').as_posix(),
    epochs=50,
    imgsz=640,
    batch=16,
    project=MODEL_TIER_1.as_posix(),
    fliplr=0.3,
)

Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_1/dataset.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_1, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=N

FileNotFoundError: ignored

In [ ]:
DATASET_TIER_2 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_2')
#DATASET_TIER_2 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_2')
imgs2 = [img.as_posix() for img in DATASET_TIER_2.joinpath('images').iterdir()]
len(imgs2)

In [ ]:
DATASET_TIER_2 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_2')
#DATASET_TIER_2 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_2')

model.predict(
    source=DATASET_TIER_2.joinpath('images').as_posix(),
    conf=0.3,
    iou=0.8,
    save=True,
    save_txt=True,
    line_thickness=2,


    )


## Работа с частью 2. 300 картинок

In [ ]:
DATASET_TIER_2 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_2')
#DATASET_TIER_2 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_2')
imgs_2 = [img.as_posix() for img in DATASET_TIER_2.joinpath('images').iterdir()]
len(imgs_2)

In [ ]:
train_2, val_2 = train_test_split(imgs_2, test_size=0.2)
print(f"Train: {len(train_2)}, Val: {len(val_2)}")

In [ ]:
DATASET_TIER_2.joinpath('train.txt').write_text('\n'.join(train_2))
DATASET_TIER_2.joinpath('val.txt').write_text('\n'.join(val_2))

In [ ]:
dataset_yaml_2 = fr'''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: {DATASET_TIER_2}  # dataset root dir
train: train.txt  # train images (relative to 'path') 128 images
val: val.txt  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: Turbine
'''
DATASET_TIER_2.joinpath('dataset.yaml').write_text(dataset_yaml_2)

Обучение модели

In [ ]:
MODEL_TIER_2 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_2')
#MODEL_TIER_2 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Models/Tier_2')
MODEL_TIER_2.mkdir(parents=True, exist_ok=True)

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
model.train(
    data=DATASET_TIER_2.joinpath('dataset.yaml').as_posix(),
    epochs=50,
    imgsz=640,
    batch=16,
    project=MODEL_TIER_2.as_posix(),
    fliplr=0.3,
)

In [ ]:

model = YOLO('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_2/train7/weights/best.pt')
#model = YOLO("/content/drive/MyDrive/Models/TurbinesDetector/Models/Tier_2/train7/weights/best.pt")

In [ ]:
DATASET_TIER_3 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_3/images')
#DATASET_TIER_3 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_3/images')

In [ ]:
imgs = [file for file in DATASET_TIER_3.iterdir()]

In [ ]:
print(f"Всего изображений: {len(imgs)}")

In [ ]:
tier_size = [800,]
tier_size = np.cumsum(tier_size)
tier_size

In [ ]:
img_tiers = np.split(imgs, tier_size) #Разбиваем массив на 2 части
len(img_tiers[0])

In [ ]:
len(img_tiers[1])

In [ ]:
for i, tier in enumerate(img_tiers):
    path_to_save = DATASET_TIERS.joinpath(f'Tier_{i+4}/images')
    path_to_save.mkdir(exist_ok=True, parents=True)
    for img in tqdm(tier):
        path_to_save.joinpath(img.name).write_bytes(img.read_bytes())

In [ ]:
DATASET_TIER_3_1 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_4')
#DATASET_TIER_3_1 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_4')

In [ ]:
imgs_3_1 = [file for file in DATASET_TIER_3_1.joinpath('images').iterdir()]
print(f"Всего изображений: {len(imgs_3_1)}")

In [ ]:
model.predict(
    source=DATASET_TIER_3_1.joinpath('images').as_posix(),
    conf=0.3,
    iou=0.8,
    save=True,
    save_txt=True,
    line_thickness=2,
    )

## Работа с частью 3. 800 картинок

In [ ]:

DATASET_TIER_3_1 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_4')
#DATASET_TIER_3_1 = Path('/content/drive/MyDrive/Models/TurbinesDetector/Tier_4')
imgs_3 = [img.as_posix() for img in DATASET_TIER_3_1.joinpath('images').iterdir()]
len(imgs_3)

In [ ]:
train_3, val_3 = train_test_split(imgs_3, test_size=0.2)
print(f"Train: {len(train_3)}, Val: {len(val_3)}")

In [ ]:
DATASET_TIER_3_1.joinpath('train.txt').write_text('\n'.join(train_3))
DATASET_TIER_3_1.joinpath('val.txt').write_text('\n'.join(val_3))

In [ ]:
dataset_yaml_3 = fr'''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: {DATASET_TIER_3_1}  # dataset root dir
train: train.txt  # train images (relative to 'path') 128 images
val: val.txt  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: Turbine
'''
DATASET_TIER_3_1.joinpath('dataset.yaml').write_text(dataset_yaml_3)

In [ ]:
MODEL_TIER_3_1 = Path('/content/drive/MyDrive/Colab Notebooks/CV/CV_HW_3/TurbinesDetector/Tier_4')
MODEL_TIER_3_1.mkdir(parents=True, exist_ok=True)

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
model.train(
    data=DATASET_TIER_3_1.joinpath('dataset.yaml').as_posix(),
    epochs=30,
    imgsz=640,
    batch=16,
    project=MODEL_TIER_3_1.as_posix(),
    fliplr=0.3,
)